# Exploratory study on existing early warning systems

## * Setup of the working environment *

### Import traditional Python packages

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from datetime import datetime as dt, timedelta, date
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import pickle
import time
import math
import json
import sys
import os

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Import custom Python modules

In [ ]:
sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from helpers.db_connector import MySQLConnector
from helpers.db_query import *

from helpers.data_process import *
from helpers.feature_extraction import *

from extractors.akpinar_et_al import AkpinarEtAl
from extractors.boroujeni_et_al import BoroujeniEtAl
from extractors.chen_cui import ChenCui
from extractors.he_et_al import HeEtAl
from extractors.lalle_conati import LalleConati
from extractors.lemay_doleck import LemayDoleck
from extractors.mbouzao_et_al import MbouzaoEtAl
from extractors.mubarak_et_al import MubarakEtAl
from extractors.wan_et_al import WanEtAl

from helpers.ml_utils import *

from helpers.time import *

## * Load the clickstream data *

Since Fall 2017, the stream of the EPFL's Linear Algebra course has been taught in a flipped format. The implementation of the flipped classroom was carried out in an incremental manner, as described below:

- **Year 2017-2018**: traditional manner (weeks 1-13) - flipped manner (week 14).
- **Year 2018-2019**: traditional manner (weeks 1-4, 10-14) - flipped manner (weeks 5-9).
- **Year 2019-2020**: traditional manner (weeks 1-4) - flipped manner (weeks 5-14).

In [ ]:
rounds = ['Y2-2018-19', 'Y3-2019-20']

### Identifying Students


The flipped course was offered only to volunteering students. The volunteers were collectively assigned into either the experimental and the control group. A stratified random sampling based on gender and the prior background (secondary educational level) of students were used.

In [ ]:
%time user_data = getUserInfo(prior_knowledge=True)

The initial data of volunteers was cleaned, and some participants were removed before we analyzed the data:
- The volunteering students who have not been graded were removed. 
- The repeating students were filtered out, where repeating students are those accessing videos in two different years. 
- The less active students, i.e., those who have provided less 60 interactions in the platform, were removed. 

Given that the Y1-2017-2018 round included only one week in a flipped classroom setting, we will remove the students of that round.  

In [ ]:
user_data = user_data[user_data['Round'].isin(rounds)]

Some of the statistics on the user data are provided below. 

In [ ]:
"Number of students:", len(user_data)

In [ ]:
sns.displot(user_data, x='Round')
plt.ylabel('Number of Students')
plt.show()

In [ ]:
sns.displot(user_data, x='Gender')
plt.ylabel('Number of Students')
plt.show()

In [ ]:
sns.displot(user_data, x='Category')
plt.ylabel('Number of Students')
plt.xticks(rotation=45)
plt.show()

### Getting Students' Records

#### Video Clickstream Records

In [ ]:
%time video_data = getVideoEventsInfo(mode='all')

In [ ]:
video_data = video_data[video_data['Round'].isin(rounds)]

In [ ]:
"Number of video events:", len(video_data)

#### Problem Clickstream Records

In [ ]:
%time problem_data = getProblemEventsInfo()

In [ ]:
problem_data = problem_data[problem_data['Round'].isin(rounds)]

In [ ]:
"Number of problem events:", len(problem_data)

#### Exam Records

In [ ]:
%time exam_data = getExamInfo()

In [ ]:
exam_data = exam_data[exam_data['Round'].isin(rounds)]

In [ ]:
exam_data = exam_data[exam_data['AccountUserID'].isin(user_data['AccountUserID'])]

In [ ]:
"Number of graded students:", len(exam_data)

#### Event Records

In [ ]:
events = video_data.append(problem_data)

In [ ]:
events['Year'] = events['Round'].apply(lambda x: int(x.split('-')[1]))

#### Course Week Column

We get the configuration file (e.g, start and end date) for each round of the course. 

In [ ]:
with open('../config/linear_algebra.json') as f:
    config = json.load(f)

We assign each video interaction to a specific week of the course, with the first week of the course round having id 0. 

In [ ]:
events['Date'] = events['TimeStamp'].apply(lambda x:string2Datetime(dt.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')))

In [ ]:
tmp_events = []
for r in rounds:
    round_events = events[events['Round'] == r]
    tmp_events.append(processWeek(round_events, 'Date', config[r.split('-')[-2]]['Start']))
events = pd.concat(tmp_events).copy()

In [ ]:
events['Week'] = events['Week'].apply(lambda x: int(x))

Then, we filter only the first *noCourseWeeks* course weeks. 

In [ ]:
events = events[events['Week'].isin(range(20))]

## * Split data in training and test sets *

In [ ]:
mode = 'random'
task = 'binary'
ratio = 80
start, end, step = 2, 16, 2

In [ ]:
weeks = np.arange(start, end, step)

In [ ]:
x_train, x_test, y_train, y_test = getTrainTestData(exam_data, mode, task, ratio / 100.0)

## * Extract and scale features from clickstreams *

In [ ]:
feature_labels = [
    AkpinarEtAl(),
    BoroujeniEtAl(),
    ChenCui(),
    HeEtAl(),
    LalleConati(),
    LemayDoleck(),
    MbouzaoEtAl(),
    MubarakEtAl(),
    # WanEtAl(),
]

First, we check whether the required feature sets have been already pre-computed. 

In [ ]:
filename = '../data/feature_sets/feature_sets_' + mode + '_' + task + '_' + str(ratio) + '_' + str(start) + '-' + str(end) + '-' + str(step) + '.pkl'

if os.path.exists(filename):
    print('Found features for this experimental setting in', filename)
    with open(filename, 'rb') as f:
        feature_sets = pickle.load(f)
else:
    feature_sets = {}

Then, for each feature set not pre-computed, the following snippet populates the corredponsing dictionary keys accordingly. 

In [ ]:
for ffunc in feature_labels:
    flabel = ffunc.getName()
    
    if not flabel in feature_sets:
        feature_sets[flabel] = {}
        
        for wid in weeks:
            feature_sets[flabel][wid] = {}
            feature_sets[flabel][wid]['train'] = []
            feature_sets[flabel][wid]['test'] = []
            scaler = StandardScaler()
            
            unactiveTrain = 0
            for uindex, uid in enumerate(x_train): 
                print('\r', 'Set:', flabel, '- Week:', wid, '- Mode: train', '- User:', uindex + 1, len(x_train), end=' ')
                udata = events[(events['AccountUserID'] == uid) & (events['Week'] < wid)]
                year = int(user_data[user_data['AccountUserID'] == uid]['Round'].values[0].split('-')[-2])
                if len(udata) > 0:
                    feature_sets[flabel][wid]['train'].append(ffunc.getUserFeatures(udata, wid, year))
                else:
                    unactiveTrain += 1
                    feature_sets[flabel][wid]['train'].append([0 for i in range(ffunc.getNbFeatures())])
            feature_sets[flabel][wid]['train'] = scaler.fit_transform(np.array(feature_sets[flabel][wid]['train']))

            print('- Unactive:', unactiveTrain, '- Active:', len(x_train) - unactiveTrain)
            
            unactiveTest = 0
            for uindex, uid in enumerate(x_test): 
                print('\r', 'Set:', flabel, '- Week:', wid, '- Mode: test', '- User:', uindex + 1, len(x_test), end=' ')
                udata = events[(events['AccountUserID'] == uid) & (events['Week'] < wid)]
                year = int(user_data[user_data['AccountUserID'] == uid]['Round'].values[0].split('-')[-2])
                if len(udata) > 0:
                    feature_sets[flabel][wid]['test'].append(ffunc.getUserFeatures(udata, wid, year))
                else:
                    unactiveTest += 1
                    feature_sets[flabel][wid]['test'].append([0 for i in range(ffunc.getNbFeatures())])
            feature_sets[flabel][wid]['test'] = scaler.transform(np.array(feature_sets[flabel][wid]['test']))
            
            print('- Unactive:', unactiveTest, '- Active:', len(x_test) - unactiveTest)

For convenience, we save the feature sets computed for the current experimental setting.

In [ ]:
if not os.path.exists(os.path.dirname(filename)):
    os.makedirs(os.path.dirname(filename))

with open(filename, 'wb') as f:
    pickle.dump(feature_sets, f, protocol=pickle.HIGHEST_PROTOCOL)

## * Define and train the predictive models *

For the prediction stage, we consider traditional shallow learning models optimized via grid search.

In [ ]:
classifiers_types = {
    'ada': AdaBoostClassifier(),
    'dt': DecisionTreeClassifier(),
    'gnb': GaussianNB(),
    'lr': LogisticRegression(),
    'mlp': MLPClassifier(),
    'knn': KNeighborsClassifier(),
    'rf': RandomForestClassifier(),
    'svm': SVC()
}

In [ ]:
classifiers_params = {
    'ada': {'n_estimators': [25, 50, 100, 200], 'algorithm': ('SAMME', 'SAMME.R'), 'learning_rate': [0.1, 1]},
    'dt': {'criterion': ('gini', 'entropy'), 'splitter': ('best', 'random'), 'max_features': ('auto', 'sqrt', 'log2')},
    'gnb': {'var_smoothing': [1e-9, 1e-7, 1e-5, 1e-3, 1e-1]},
    'lr': {'penalty': ('l1', 'l2', 'elasticnet', 'none'), 'tol': [1e-4, 1e-5], 'C': [1.0, 0.5], 'solver': ('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'), 'multi_class': ('auto', 'ovr', 'multinomial')},
    'mlp': {'activation': ('identity', 'logistic', 'tanh', 'relu'), 'solver': ('lbfgs', 'sgd', 'adam'), 'hidden_layer_sizes': [(8,), (16, 8), (32, 16, 8)]},
    'knn': {'n_neighbors': [5, 10, 50, 100], 'weights': ('uniform', 'distance'), 'algorithm': ('auto', 'ball_tree', 'kd_tree', 'brute')},
    'rf': {'n_estimators': [25, 50, 100, 200], 'criterion': ('gini', 'entropy'), 'max_features': ('auto', 'sqrt', 'log2')},
    'svm': {'C': [1.0, 0.5], 'kernel': ('linear', 'poly', 'rbf', 'sigmoid'), 'gamma': ('scale', 'auto'), 'shrinking': (True, False)}
}

For convenience, we load the trained models, in case they have been already computed under the same experimental setting. 

In [ ]:
filename = '../data/trained_models/trained_models_' + mode + '_' + task + '_' + str(ratio) + '_' + str(start) + '-' + str(end) + '-' + str(step) + '.pkl'

if os.path.exists(filename):
    print('Found trained_models for this experimental setting in', filename)
    with open(filename, 'rb') as f:
        trained_models = pickle.load(f)
else:
    trained_models = {}

Once the prediction stage is well set, we run the training procedure for each of the considered models. 

In [ ]:
for ffunc in feature_labels:
    flabel = ffunc.getName()
    
    if not flabel in trained_models:
        trained_models[flabel] = {}
        
        for wid in weeks:
            trained_models[flabel][wid] = {}
            print('Model:', flabel, '- Week:', wid, '- Algorithms:', end=' ')
            
            for mid in classifiers_types.keys(): 
                print(mid, end=' ')
                trained_models[flabel][wid][mid] = GridSearchCV(classifiers_types[mid], classifiers_params[mid])
                trained_models[flabel][wid][mid].fit(feature_sets[flabel][wid]['train'], y_train)
            
            print()

We list the parameters selected by the grid search procedure, investigating whether they remain stable across weeks. 

In [ ]:
for ffunc in feature_labels:
    flabel = ffunc.getName()
    print('Feature set:', flabel)
    for wid in weeks:
        print('> Week:', wid)
        for mid in classifiers_types.keys():
            print('>> Model:', mid, '-', trained_models[flabel][wid][mid].best_params_)

For convenience, we save the trained models computed for the current experimental setting.

In [ ]:
if not os.path.exists(os.path.dirname(filename)):
    os.makedirs(os.path.dirname(filename))

with open(filename, 'wb') as f:
    pickle.dump(trained_models, f, protocol=pickle.HIGHEST_PROTOCOL)

## * Define and compute the evaluation metrics *

In [ ]:
def tn(y_true, y_pred):
    return confusion_matrix(y_true, y_pred).ravel()[0]

def fp(y_true, y_pred):
    return confusion_matrix(y_true, y_pred).ravel()[1]

def fn(y_true, y_pred):
    return confusion_matrix(y_true, y_pred).ravel()[2]

def tp(y_true, y_pred):
    return confusion_matrix(y_true, y_pred).ravel()[3]

def tpr(y_true, y_pred): 
    return tp(y_true, y_pred) / (tp(y_true, y_pred) + fn(y_true, y_pred))

def tnr(y_true, y_pred):
    return tn(y_true, y_pred) / (tn(y_true, y_pred) + fp(y_true, y_pred))

def fpr(y_true, y_pred): 
    return fp(y_true, y_pred) / (fp(y_true, y_pred) + tn(y_true, y_pred))

def fnr(y_true, y_pred): 
    return fn(y_true, y_pred) / (tp(y_true, y_pred) + fn(y_true, y_pred))

def eer(y_true, y_pred):
    return np.mean([fnr(y_true, y_pred), fpr(y_true, y_pred)])

Traditional metrics of effectiveness are considered for the prediction stage. Given that early warning systems should take particular care of the false negatives instances, the False Positive Rate will represent an essential measure of effectiveness.   

In [ ]:
evaluation_metrics = {
    'Acc': accuracy_score,
    'F1': f1_score,
    'P': precision_score, 
    'R': recall_score,
    'TP': tp,
    'FN': fn,
    'TN': tn,
    'FP': fp,
    'TPR': tpr,
    'TNR': tnr,
    'FPR': fpr, 
    'FNR': fnr,
    'EER': eer
}

In [ ]:
results = {}
for ffunc in feature_labels:
    flabel = ffunc.getName()
    results[flabel] = {}
    for wid in weeks:
        print(flabel, wid, end='\t')
        results[flabel][wid] = {}
        for mid in classifiers_types.keys(): 
            print(mid, end=' ')
            results[flabel][wid][mid] = {}
            clf = trained_models[flabel][wid][mid]
            for emid, mfunc in evaluation_metrics.items():
                results[flabel][wid][mid][emid] = mfunc(y_test, clf.predict(feature_sets[flabel][wid]['test']))
        print()

## * Show and discuss the results *

In [ ]:
lst_data = []
lst_name = []
for flabel in results.keys():
    for wid in results[flabel].keys():
        for mid in results[flabel][wid].keys():
            lst_data.append([wid, flabel, mid] + [value for _, value in results[flabel][wid][mid].items()])  
            lst_name = ['week', 'set', 'clf'] + [emid for emid, _ in results[flabel][wid][mid].items()]

In [ ]:
df_results = pd.DataFrame(lst_data, columns = lst_name)

In [ ]:
df_results[df_results['week'] == 2].set_index(['week', 'set', 'clf'])

In [ ]:
df_results[df_results['week'] == 14].set_index(['week', 'set', 'clf'])

In [ ]:
def plot_metrics_along_weeks(emid):
    tnr_time = {}
    for ffunc in feature_labels:
        flabel = ffunc.getName()
        tnr_time[flabel] = {}
        for wid in weeks:
            for mid in classifiers_types.keys(): 
                if not mid in tnr_time[flabel]:
                    tnr_time[flabel][mid] = []
                tnr_time[flabel][mid].append(results[flabel][wid][mid][emid])

    plt.figure(num=None, figsize=(20, 6), dpi=80, facecolor='w', edgecolor='k')
    for i, ffunc in enumerate(feature_labels):
        flabel = ffunc.getName()
        plt.subplot(len(feature_labels) // 2, 2, i + 1)
        plt.title(flabel)
        colors = [plt.cm.rainbow(x) for x in np.linspace(0, 1, len(classifiers_types))]
        for j, mid in enumerate(classifiers_types.keys()): 
            plt.plot(weeks, tnr_time[flabel][mid], 'o-', lw=1, color=colors[j], label=mid)
        plt.ylim([0.2, 0.7])
        plt.xlim([weeks[0], weeks[-1]])
        plt.xlabel('Course week')
        plt.ylabel(emid)
        plt.grid(axis='y')
        plt.legend(loc='upper left')

In [ ]:
plot_metrics_along_weeks('EER')